# MNIST-Clean
### Image Translation

#### Imports

In [1]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import *
from cox.utils import Parameters

from robustness import model_utils, datasets, train, defaults
from robustness.model_utils import make_and_restore_model
from DatasetsNew import MNIST

import cox.store
import torch as ch
import DatasetsNew
import defaultsNew
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
import copy
import numpy as np

import pickle
import sys
import psutil
import gc


from label_maps import CLASS_DICT
from user_constants import DATA_PATH_DICT

#### DATASET CONSTANTS 

In [2]:
NUM_WORKERS = 8
BATCH_SIZE = 100

In [3]:
DATA = 'MNIST' # Choices: ['H2Z', 'A2O', 'S2W']

#### Load dataset

In [4]:
# Load dataset
dataset_function = getattr(DatasetsNew, 'MNIST')
DATA_PATH_DICT[DATA]
dataset = dataset_function(DATA_PATH_DICT[DATA])
_, test_loader = dataset.make_loaders(workers=NUM_WORKERS,
                                      batch_size=BATCH_SIZE, 
                                      data_aug=False)
data_iterator = enumerate(test_loader.dataset)

==> Preparing dataset mnist..


In [11]:
kwargs = {
        'constraint':'2',
        'eps': 100,
        'step_size': 0.25,
        'iterations': 50,
        'do_tqdm': True,
        'targeted': True,
}


### Translation for Clean MNIST model

In [14]:
model_clean, _ = make_and_restore_model(arch='resnet18', dataset=dataset,resume_path='models/MNIST_Clean_100_epochs_checkpoint.pt.best',parallel=False) 
model_clean.eval()
print()

=> loading checkpoint 'models/MNIST_Clean_100_epochs_checkpoint.pt.best'
=> loaded checkpoint 'models/MNIST_Clean_100_epochs_checkpoint.pt.best' (epoch 91)



In [17]:
_, train_loader = dataset.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE)
_, (imgs, label) = next(enumerate(train_loader))

for targ_lbl in range(10):
# for targ_lbl in [1]:
    targ = []


    for i in range(BATCH_SIZE):
        targ.append(targ_lbl)
    targ = ch.tensor(targ)

    _, img_translated = model_clean(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)

    for i in range(4):
        _, (imgs, label) = next(enumerate(train_loader))
        targ = []
        for i in range(BATCH_SIZE):
            targ.append(targ_lbl)
        targ = ch.tensor(targ)

        _, img_translated_new = model_clean(imgs.cuda(), targ.cuda(), make_adv=True, **kwargs)
        img_translated = ch.cat((img_translated, img_translated_new), 0)
    #     clean_img_ch = ch.cat((clean_img_ch, imgs[i].reshape(1,3,32,32)), 0)

    print(img_translated.size())
    
    
    filename = "saved_pickles/clean-model-img-translated/img_translated_from_dataset_" + str(targ_lbl) + ".pkl"
    
    with open(filename, 'wb') as handle:
        pickle.dump(img_translated, handle)
    print("saved: ", filename)

==> Preparing dataset mnist..


Current loss: 7.917923450469971:   4%|▍         | 2/50 [00:00<00:02, 19.43it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_0.pkl


Current loss: 8.078473091125488:   4%|▍         | 2/50 [00:00<00:02, 19.10it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_1.pkl


...

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_7.pkl


Current loss: 8.492318153381348:   6%|▌         | 3/50 [00:00<00:02, 20.12it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_8.pkl


Current loss: 0.00020975113147869706: 100%|██████████| 50/50 [00:02<00:00, 19.72it/s]

torch.Size([500, 1, 28, 28])
saved:  saved_pickles/clean-model-img-translated/img_translated_from_dataset_9.pkl
